In [1]:
import gradio as gr
import pandas as pd
import hopsworks
import joblib
import datetime
import os
import requests

c:\Users\Benj3\anaconda3\envs\energy\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
project = hopsworks.login()
fs = project.get_feature_store()

mr = project.get_model_registry()
model = mr.get_model("electricity_price_prediction_model", version=1)
model_dir = model.download()
model = joblib.load(model_dir + "/dk_electricity_model.pkl")

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/556180
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


In [ ]:
def predict():
    today = get_date()
    temp = get_temp(today)
    df = pd.DataFrame({"date": [today], "temperature": [temp]})
    df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
    df['day'] = df['date'].dt.dayofweek
    df['month'] = df['date'].dt.month
    holidays = calendar().holidays(start=df['date'].min(), end=df['date'].max())
    df['holiday'] = df['date'].isin(holidays).astype(int)

    demand = model.predict(df.drop(columns=['date']))[0]
    return [today, temp, demand]
